# Floating Coupon Bond

The goal of the project is to compute the market value of a portfolio of floating coupon bonds taking into account the issuer credit risk. The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* A set of forward libor rates and their fixing dates
* A set of survival probabilities and the recovery rate of the issuer
* The static data of the pool of bonds to price (nominal, start date, end date, current coupon, margin, payment frequency)

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

### Inputs

In [1]:
from datetime import date 

today = date(2013, 10, 31)

libor_tenor = 6 

ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

survival_probabilities = [
    {'date': date(2014, 12, 20), 'ndp': 0.972159727015014},
    {'date': date(2015, 12, 20), 'ndp': 0.942926329174406},
    {'date': date(2016, 12, 20), 'ndp': 0.913448056250137},
    {'date': date(2018, 12, 20), 'ndp': 0.855640452819766},
    {'date': date(2023, 12, 20), 'ndp': 0.732687779675469},
    {'date': date(2033, 12, 20), 'ndp': 0.539046016487758},
]

euribor_6m = [
    {'date': date(2013, 10, 31), 'rate': 0.005},
    {'date': date(2014, 4, 30), 'rate': 0.0058},
    {'date': date(2014, 10, 30), 'rate': 0.0066},
    {'date': date(2015, 4, 30), 'rate': 0.0074},
    {'date': date(2015, 10, 30), 'rate': 0.0082},
    {'date': date(2016, 4, 30), 'rate': 0.009},
    {'date': date(2016, 10, 30), 'rate': 0.0098},
    {'date': date(2017, 4, 30), 'rate': 0.0106},
    {'date': date(2017, 10, 30), 'rate': 0.0114},
    {'date': date(2018, 4, 30), 'rate': 0.0122},
    {'date': date(2018, 10, 30), 'rate': 0.013},
    {'date': date(2019, 4, 30), 'rate': 0.0138},
    {'date': date(2019, 10, 30), 'rate': 0.0146},
    {'date': date(2020, 4, 30), 'rate': 0.0154},
    {'date': date(2020, 10, 30), 'rate': 0.0162},
    {'date': date(2021, 4, 30), 'rate': 0.017},
    {'date': date(2021, 10, 30), 'rate': 0.0178},
    {'date': date(2022, 4, 30), 'rate': 0.0186},
    {'date': date(2022, 10, 30), 'rate': 0.0194},
    {'date': date(2023, 4, 30), 'rate': 0.0202},
    {'date': date(2023, 10, 30), 'rate': 0.021},
    {'date': date(2024, 4, 30), 'rate': 0.0218},
    {'date': date(2024, 10, 30), 'rate': 0.0226},
    {'date': date(2025, 4, 30), 'rate': 0.0234},
    {'date': date(2025, 10, 30), 'rate': 0.0242},
    {'date': date(2026, 4, 30), 'rate': 0.025},
    {'date': date(2026, 10, 30), 'rate': 0.0258},
    {'date': date(2027, 4, 30), 'rate': 0.0266},
    {'date': date(2027, 10, 30), 'rate': 0.0274},
    {'date': date(2028, 4, 30), 'rate': 0.0282},
]

bonds_to_price = [
    {'nominal': 4972284.02, 'start_date': date(2010, 3, 1), 'end_date': date(2021, 8, 1),
     'current_coupon': 0.0275, 'margin': 0.0175, 'recovery': 0.2},
    {'nominal': 7344328.27, 'start_date': date(2009, 7, 1), 'end_date': date(2022, 7, 1), 'current_coupon': 0.0225, 'margin': 0.0175, 'recovery': 0.2},
    {'nominal': 7172290.19, 'start_date': date(2013, 1, 1), 'end_date': date(2017, 9, 1), 'current_coupon': 0.035, 'margin': 0.01, 'recovery': 0.4},
    {'nominal': 7065224.23, 'start_date': date(2010, 3, 1), 'end_date': date(2014, 10, 1), 'current_coupon': 0.018, 'margin': 0.01, 'recovery': 0.4},
    {'nominal': 5256452.14, 'start_date': date(2011, 7, 1), 'end_date': date(2016, 4, 1), 'current_coupon': 0.039, 'margin': 0.01, 'recovery': 0.4},
    {'nominal': 2689680.89, 'start_date': date(2009, 9, 1), 'end_date': date(2024, 7, 1), 'current_coupon': 0.032, 'margin': 0.01, 'recovery': 0.6},
    {'nominal': 3593518.71, 'start_date': date(2010, 7, 1), 'end_date': date(2019, 2, 1), 'current_coupon': 0.032, 'margin': 0.01, 'recovery': 0.6},
    {'nominal': 6993589.53, 'start_date': date(2011, 1, 1), 'end_date': date(2018, 11, 1), 'current_coupon': 0.021, 'margin': 0.01, 'recovery': 0.6},
    {'nominal': 6684377.52, 'start_date': date(2009, 9, 1), 'end_date': date(2021, 9, 1), 'current_coupon': 0.019, 'margin': 0.01, 'recovery': 0.6},
    {'nominal': 6896199.04, 'start_date': date(2010, 7, 1), 'end_date': date(2018, 11, 1), 'current_coupon': 0.0235, 'margin': 0.0135, 'recovery': 0.4},
    {'nominal': 2587984.6, 'start_date': date(2011, 10, 1), 'end_date': date(2020, 10, 1), 'current_coupon': 0.016, 'margin': 0.01, 'recovery': 0.4},
    {'nominal': 3621656.1, 'start_date': date(2012, 6, 1), 'end_date': date(2016, 7, 1), 'current_coupon': 0.0325, 'margin': 0.0135, 'recovery': 0.4},
    {'nominal': 3146567.47, 'start_date': date(2011, 6, 1), 'end_date': date(2022, 3, 1), 'current_coupon': 0.019, 'margin': 0.009, 'recovery': 0.2},
    {'nominal': 6452721.61, 'start_date': date(2009, 4, 1), 'end_date': date(2019, 4, 1), 'current_coupon': 0.029, 'margin': 0.009, 'recovery': 0.2},
    {'nominal': 3418346.24, 'start_date': date(2010, 5, 1), 'end_date': date(2016, 1, 1), 'current_coupon': 0.026, 'margin': 0.009, 'recovery': 0.2},
]

## Solution
The first step is to determine the discoutn curve using the bootstrap method.
So define a set of OIS according to the market quotes given in inputs.

In [2]:
from finmarkets import OvernightIndexSwap, generate_swap_dates
#from project_input import today, ois_quotes

ois_pillar_dates = [today]

swaps = []

for quote in ois_quotes:
    
    Swap = OvernightIndexSwap( 
                1e6, 
                generate_swap_dates(today,quote['maturity']),
                quote['rate']
            )
    
    swaps.append(Swap)
    ois_pillar_dates.append(Swap.payment_dates[-1])
    
ois_pillar_dates = sorted(ois_pillar_dates)
n_df_vector = len(ois_pillar_dates)

Then define the objective function with a discount curve with unknown discount factors $\vec{x}$ and the squared sum of the swap NPVs to minimize.

In [3]:
from finmarkets import DiscountCurve

def objective_function(x): 
    
    curve = DiscountCurve(today, ois_pillar_dates, x)
    
    sum_sq = 0.0 
    
    for swap in swaps: 
        sum_sq += swap.npv(curve)**2
        
    return sum_sq

Set initial values and boundaries of the unknown factors (remember that the first factor has to be 1 since refers to today's discount).

In [4]:
from scipy.optimize import minimize 

x0 = [1.0 for i in range(len(ois_pillar_dates))]

bounds = [(0.01,100.0) for i in range(len(ois_pillar_dates))]
bounds[0]= (1.0,1.0)

result = minimize(objective_function, x0, bounds = bounds)

Check the results to be sure the minimization worked...

In [5]:
print (result)

      fun: 0.0006561289593822537
 hess_inv: <31x31 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.79535027e+05, -2.83218934e-04, -3.24325939e-04, -3.24159579e-04,
       -3.11566571e-04, -3.47202302e-04, -3.37286557e-04, -2.87379278e-04,
       -3.00611012e-04, -1.85489428e-04, -1.04252349e-04, -1.83492739e-04,
       -2.79941769e-03,  2.00371273e-03,  2.00847496e-03,  1.81515215e-03,
        2.94133554e-04,  2.56011760e-03,  2.71539768e-03,  5.59571356e-04,
       -1.80683901e-03, -2.87603688e-03, -2.12279051e-03, -6.51237224e-04,
        8.25915508e-04,  1.54731075e-03,  1.69996320e-03, -1.17257943e-03,
        1.79882329e-03,  2.55535285e-04, -1.25285385e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 448
      nit: 11
   status: 0
  success: True
        x: array([1.        , 0.99991167, 0.99980687, 0.99970619, 0.99961015,
       0.99950111, 0.999392  , 0.99928207, 0.99916713, 0.99903027,
       0.998895  , 0.99874906, 0.99860278, 0.9980

In [6]:
print ("Initial value of obj_func {}".format(objective_function(x0)))
print ("Final value of obj_func {}".format(objective_function(result.x)))

Initial value of obj_func 1398807951100.5298
Final value of obj_func 0.0006561289593822537


Finally create the discount curve to be used in the exercise.

In [7]:
from finmarkets import DiscountCurve
from datetime import date 

eonia_curve = DiscountCurve(today, 
                            ois_pillar_dates, 
                            result.x)

eonia_curve.df(date(2030,10,31))

0.6710110388246098

Our bonds are indexed to a LIBOR 6m curve given in input so we need to create a `ForwardRateCurve` object to use it.

In [8]:
from finmarkets import ForwardRateCurve
from datetime import date

forward_rate_curve= ForwardRateCurve(
                    [item['date'] for item in euribor_6m],
                    [item['rate'] for item in euribor_6m])               
                
forward_rate_curve.forward_rate(date(2025,7,30))

0.023797814207650272

In a similar way we need to create a `CreditCurve` from the inputs.

In [9]:
from finmarkets import CreditCurve
from datetime import date

pillar_ndps = [1.0]
ndps_pillar_dates = [today]

for ndps in survival_probabilities:
    pillar_ndps.append(ndps['ndp'])
    
for dates in survival_probabilities:
    ndps_pillar_dates.append(dates['date'])
    
credit_curve = CreditCurve(ndps_pillar_dates,
                           pillar_ndps)

credit_curve.ndp(date(2025,10,31))

0.691942752218531

Now that we have all the ingredients we can compute the market values of the portfolio of bonds.

In [10]:
from datetime import date 
from dateutil.relativedelta import relativedelta 

class BondPricer: 
    
    def __init__(self,pricing_date, nominal, start_date, end_date,
                 current_coupon, margin, recovery, tenor_months=6):
        
        self.pricing_date = pricing_date 
        self.nominal = nominal
        self.start_date = start_date 
        self.end_date = end_date 
        self.current_coupon = current_coupon
        self.margin = margin 
        self.recovery = recovery 
        self.tenor_months = tenor_months 
        

    def payment_dates(self):
        dates = []
        future_dates = []

        n_months = ((self.end_date.year - self.start_date.year) * 12) + \
                    (self.end_date.month - self.start_date.month)

        for n in range(0, n_months, self.tenor_months):
            dates.append(self.start_date + relativedelta(months=n))
        dates.append(self.end_date)

        for i in range(len(dates)):

            if dates[i] > self.pricing_date: 
                future_dates.append(dates[i])

        return future_dates
        
    def mark_to_market(self, discounting_curve, forward_curve, credit_curve):
        
        bond_dates = self.payment_dates()
        npv_coupons_if_no_default = 0
        recovery_if_early_default = 0 
    
        for i in range(1, len(bond_dates)):
            if bond_dates[i] > today:
                npv_coupons_if_no_default += self.nominal *            
                    (((bond_dates[i] - bond_dates[i-1]).days / 360) *
                        discounting_curve.df(bond_dates[i]) *
                        (forward_curve.forward_rate(bond_dates[i-1]) + self.margin) * 
                        credit_curve.ndp(bond_dates[i]))
                    
        notional_reimbursement_if_no_default = self.nominal * 
                    (discounting_curve.df(bond_dates[-1])*
                     credit_curve.ndp(bond_dates[-1]))
        
        
        first_coupon = (((bond_dates[0] - today).days / 360)*
                          discounting_curve.df(bond_dates[0]) *
                          self.current_coupon * credit_curve.ndp(bond_dates[0]))

        npv_coupons_if_no_default += first_coupon
        
        
        
        
        
        d= self.pricing_date
    
        while d <= bond_dates[-1]:

            recovery_if_early_default += ((discounting_curve.df(d)) *
                                         (credit_curve.ndp(d) - 
                                          credit_curve.ndp(d+relativedelta(days=1))))
            d += relativedelta(days=1)  

        recovery_if_early_default = recovery_if_early_default * self.recovery * self.nominal
        
        MTM =  npv_coupons_if_no_default + notional_reimbursement_if_no_default + recovery_if_early_default
        
        return MTM